<a href="https://colab.research.google.com/github/neild0/StyleDrop-PyTorch-Interactive/blob/main/styledrop_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/zideliu/StyleDrop-PyTorch
!pip install gdown accelerate==0.12.0 absl-py ml_collections einops wandb ftfy==6.1.1 transformers==4.23.1 loguru webdataset==0.2.5 gradio xformers

Cloning into 'StyleDrop-PyTorch'...
remote: Enumerating objects: 364, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 364 (delta 42), reused 32 (delta 14), pack-reused 289
Receiving objects: 100% (364/364), 44.00 MiB | 25.34 MiB/s, done.
Resolving deltas: 100% (96/96), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.0/144.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 k

In [2]:
!git lfs install
!git clone https://huggingface.co/nzl-thu/MUSE
!mv MUSE/assets/* StyleDrop-PyTorch/assets/
!rm -rf MUSE

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'MUSE'...
remote: Enumerating objects: 25, done.
remote: Total 25 (delta 0), reused 0 (delta 0), pack-reused 25
Unpacking objects: 100% (25/25), 3.01 KiB | 513.00 KiB/s, done.
Filtering content: 100% (12/12), 9.08 GiB | 47.35 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	assets/ckpts/cc3m-285000.ckpt/optimizer.pth

See: `git lfs help smudge` for more details.


In [3]:
!gdown --id '13S_unB87n6KKuuMdyMnyExW0G1kplTbP' --output StyleDrop-PyTorch/assets/vqgan_jax_strongaug.ckpt

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=13S_unB87n6KKuuMdyMnyExW0G1kplTbP
To: /content/StyleDrop-PyTorch/assets/vqgan_jax_strongaug.ckpt
100% 218M/218M [00:03<00:00, 57.3MB/s]


In [4]:
!python StyleDrop-PyTorch/extract_empty_feature.py

2023-07-08 06:28:02.651 | DEBUG    | open_clip.transformer:__init__:314 - xattn in transformer of CLIP is True
2023-07-08 06:28:19.731 | DEBUG    | open_clip.transformer:__init__:314 - xattn in transformer of CLIP is True
^C


In [5]:
import os
import subprocess

if 'EVAL_CKPT' in os.environ:
    del os.environ['EVAL_CKPT']

if 'ADAPTER' in os.environ:
    del os.environ['ADAPTER']

os.environ['OUTPUT_DIR'] = "training_data"

subprocess.run(['accelerate', 'launch', '--num_processes', '8', '--mixed_precision', 'fp16', 'StyleDrop-PyTorch/train_t2i_custom_v2.py', '--config=StyleDrop-PyTorch/configs/custom.py'])

KeyboardInterrupt: ignored

In [6]:
!accelerate launch --num_processes 8 --mixed_precision fp16 StyleDrop-PyTorch/train_t2i_custom_v2.py --config=StyleDrop-PyTorch/configs/custom.py


2023-07-08 06:40:01.588207: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-08 06:40:03.807742: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_machines` was set to a value of `1`
	`--num_cpu_threads_per_process` was set to `1` to improve out-of-box performance
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2023-07-08 06:40:13.252016: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-tim